# Census and GeoJSON Data EDA

The goal of this notebook is to obtain and organize the following county-level data:

- nominal data: state, county, fips
- census data: 
    - total population
    - ethnic population(s)
    - voting statistics
    - median income
    - educational attainment
- geographic data (from GeoJSON): 
    - census area
    - latitude/longitude

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
import json

import itertools

from time import time
from datetime import datetime, timedelta

from shapely.geometry import Polygon
from shapely.geometry import asShape
from geojson import Feature

# 1. import census data from `census.gov`

2019 population estimates can be collected from [census.gov](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/). For the most current estimates, we will only save data from `YEAR == 12` ([data dictionary](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf)).

In [2]:
with urlopen('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/cc-est2019-alldata.csv') as response:
    dem_df = pd.read_csv(
        response,
        encoding='latin-1',        # to avoid unicode error
        dtype={'STATE':'str',
               'COUNTY':'str'}
    )
dem_df = dem_df.drop(columns='SUMLEV')         # SUMLEV == 50 for the 50 US states
dem_df = dem_df.loc[(dem_df['YEAR'] == 12) & (dem_df['AGEGRP'] == 0)]    # population estimate for 2019
dem_df = dem_df.drop(columns=['YEAR', 'AGEGRP'])
dem_df.head()

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
209,01,001,Alabama,Autauga County,55869,27092,28777,20878,21729,5237,...,778,687,89,93,40,27,15,19,16,11
437,01,003,Alabama,Baldwin County,223234,108247,114987,94810,100388,9486,...,5144,4646,268,281,264,197,69,65,55,35
665,01,005,Alabama,Barbour County,24686,13064,11622,6389,5745,6311,...,509,408,63,50,61,26,1,0,14,8
893,01,007,Alabama,Bibb County,22394,11929,10465,8766,8425,2941,...,291,253,32,19,6,15,5,1,17,3
1121,01,009,Alabama,Blount County,57826,28472,29354,27258,28154,516,...,2794,2516,76,58,67,66,18,21,34,21


In [3]:
dem_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 209 to 716357
Data columns (total 77 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATE         3142 non-null   object
 1   COUNTY        3142 non-null   object
 2   STNAME        3142 non-null   object
 3   CTYNAME       3142 non-null   object
 4   TOT_POP       3142 non-null   int64 
 5   TOT_MALE      3142 non-null   int64 
 6   TOT_FEMALE    3142 non-null   int64 
 7   WA_MALE       3142 non-null   int64 
 8   WA_FEMALE     3142 non-null   int64 
 9   BA_MALE       3142 non-null   int64 
 10  BA_FEMALE     3142 non-null   int64 
 11  IA_MALE       3142 non-null   int64 
 12  IA_FEMALE     3142 non-null   int64 
 13  AA_MALE       3142 non-null   int64 
 14  AA_FEMALE     3142 non-null   int64 
 15  NA_MALE       3142 non-null   int64 
 16  NA_FEMALE     3142 non-null   int64 
 17  TOM_MALE      3142 non-null   int64 
 18  TOM_FEMALE    3142 non-null   int64 
 19  WA

Notice that county names provided by the US census contain descriptive terms, such as 'County', whereas the NYTimes data does not.

In [4]:
# remove descriptive terms from county names, will use on other dataframes
def remove_county_terms(data, county_col):
    county_terms = ['County', 'Parish', 'Municipality']
    for term in county_terms:
        data[county_col] = data[county_col].str.replace(' ' + term, '')
    return data

In [5]:
# rename columns to better-match nytimes data (and personal preference)
dem_df.rename(
    columns={
        'STATE':'statefips',
        'COUNTY':'countyfips',
        'STNAME':'state',
        'CTYNAME':'county',
        'TOT_POP': 'total_pop'
    }, inplace=True
)
dem_df.columns = dem_df.columns.str.lower()

# nytimes fips is 5-digit combo of state and county fips
dem_df['fips'] = dem_df['statefips'] + dem_df['countyfips']
dem_df = dem_df.drop(columns=['statefips', 'countyfips'])

dem_df = remove_county_terms(dem_df, 'county')

dem_df.pipe(remove_county_terms, 'county')
dem_df.head()

,state,county,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,...,hwac_female,hbac_male,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,fips
209,Alabama,Autauga,55869,27092,28777,20878,21729,5237,6000,121,...,687,89,93,40,27,15,19,16,11,01001
437,Alabama,Baldwin,223234,108247,114987,94810,100388,9486,10107,903,...,4646,268,281,264,197,69,65,55,35,01003
665,Alabama,Barbour,24686,13064,11622,6389,5745,6311,5595,103,...,408,63,50,61,26,1,0,14,8,01005
893,Alabama,Bibb,22394,11929,10465,8766,8425,2941,1822,53,...,253,32,19,6,15,5,1,17,3,01007
1121,Alabama,Blount,57826,28472,29354,27258,28154,516,462,192,...,2516,76,58,67,66,18,21,34,21,01009


## check county names against NYTimes data

We eventually need to merge with the NYTimes data, so let's see how they match with each other:

In [6]:
with urlopen('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv') as response:
    nyt_df = pd.read_csv(
        response,
        dtype={'fips':'str'}
    )
nyt_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [7]:
county_diffs = list(set(nyt_df['county']) - set(dem_df['county']))
len(county_diffs)

84

In [ ]:
sorted([str(f) for f in county_diffs])

As expected, the census county data is missing all municipios from [Puerto Rico](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-puerto-rico-municipios.html) (`fips == 72`) as well as a couple from the Northern Mariana Islands (`fips == 69`), so we need to append that data to `pop_df`.

## make rows for New York City, Kansas City, and Joplin

Since the NYTimes dataset treats `New York City`, `Kansas City`, and `Joplin` [as their own entities](https://github.com/nytimes/covid-19-data#geographic-exceptions), we need to add them to our population dataframe.

### New York City

`New York City` is the combination of these five counties, [which are coterminous with the five boroughs](https://en.wikipedia.org/wiki/New_York_City#Boroughs):

- Bronx (36005)
- Kings (36047)
- New York (36061)
- Queens (36081)
- Richmond (36085)

We will arbitrarily assign the `fips` as `36NYC`.

In [8]:
boroughs = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']

nyc_dem_df = dem_df[(dem_df['state'] == 'New York') & (dem_df['county'] == boroughs[0])].select_dtypes(include='number')

for b in boroughs[1:]:
    nyc_dem_df += dem_df[(dem_df['state'] == 'New York') & (dem_df['county'] == b)].select_dtypes(include='number').values

nyc_dem_df['state'] = 'New York'
nyc_dem_df['county'] = 'New York City'
nyc_dem_df['fips'] = '36NYC'
nyc_dem_df.head()

,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,ia_female,aa_male,...,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,state,county,fips
417449,8336817,3978439,4358378,2145238,2247804,1046937,1247761,57897,58600,597346,...,294492,65574,67225,21187,22500,8967,9468,New York,New York City,36NYC


In [9]:
dem_df = dem_df.append(nyc_dem_df, ignore_index=True)
dem_df[dem_df['fips'] == '36NYC']

,state,county,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,...,hwac_female,hbac_male,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,fips
3142,New York,New York City,8336817,3978439,4358378,2145238,2247804,1046937,1247761,57897,...,922001,257362,294492,65574,67225,21187,22500,8967,9468,36NYC


In [10]:
dem_df['white'] = (dem_df['nhwa_male']+dem_df['nhwa_female'])
dem_df['black'] = (dem_df['nhba_male']+dem_df['nhba_female'])
dem_df['asian'] = (dem_df['nhaa_male']+dem_df['nhaa_female'])
dem_df['hispanic'] = (dem_df['h_male']+dem_df['h_female'])

In [11]:
pop_cols = ['state', 'county', 'total_pop', 'fips', 'white', 'black', 'asian', 'hispanic']
pop_df = dem_df[pop_cols]
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic
3138,Wyoming,Teton,23464,56039,19000,145,378,3554
3139,Wyoming,Uinta,20226,56041,17657,126,92,1871
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108
3141,Wyoming,Weston,6927,56045,6236,45,113,285
3142,New York,New York City,8336817,36NYC,2681976,1825848,1228598,2423590


### Kansas City and Joplin

Kansas City and Joplin both refer to cities that cross county borders in Missouri. Therefore, we have to get our information from [census.gov quickfacts](https://www.census.gov/quickfacts).

We'll use `29KAN`, and `29JOP` as our `fips` for these two cities.

In [12]:
mo_pop_df = pd.DataFrame(
    [['Missouri',
      'Kansas City',
      495_327,
      '29KAN',
      int(0.601*495_327),
      int(0.290*495_327),
      int(0.027*495_327),
      int(0.102*495_327)],
     ['Missouri',
      'Joplin',
      50_925,
      '29JOP',
      int(0.876*50_925),
      int(0.032*50_925),
      int(0.019*50_925),
      int(0.048*50_925)
     ]]
    , columns=pop_cols)
mo_pop_df

,state,county,total_pop,fips,white,black,asian,hispanic
0,Missouri,Kansas City,495327,29KAN,297691,143644,13373,50523
1,Missouri,Joplin,50925,29JOP,44610,1629,967,2444


In [13]:
pop_df = pop_df.append(mo_pop_df, ignore_index=True)
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108
3141,Wyoming,Weston,6927,56045,6236,45,113,285
3142,New York,New York City,8336817,36NYC,2681976,1825848,1228598,2423590
3143,Missouri,Kansas City,495327,29KAN,297691,143644,13373,50523
3144,Missouri,Joplin,50925,29JOP,44610,1629,967,2444


# 2. import geojson for boundaries and census areas

In [14]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    county_json = json.load(response)

In [15]:
# inspect structure of geojson
county_json['features'][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

## add areas to `county_json`

In [16]:
fips_to_add_to_json = list(set(nyt_df['fips']) - set([f['id'] for f in county_json['features']]))
fips_to_add_to_json

[nan, '69110', '78010', '02158', '78030', '46102', '69120', '78020']

The `plotly` county GeoJSON dataset is missing Kusilvak Census Area (`'02158'`) and Oglala Lakota County(`'46102'`), in addition to the three cities included in the NYTimes data (New York City, Kansas City, Joplin). 

**FUTURE WORK**: we will ignore the following entries in `nyt_df`:
- Northern Mariana Islands (`69xxx`)
- US Virgin Islands (`78xxx`)
- state totals from an unknown county source (`nan`)

GeoJSON data for these five areas compiled from [nomanatim](https://nominatim.openstreetmap.org/) and [polygons](http://polygons.openstreetmap.fr/):
- Search for the area at [nomanatim](https://nominatim.openstreetmap.org/).
- Select `details` from the relevant entry.
- Copy the numeric `code` under `OSM`, ignoring "relation". Eg. for New York City, copy `175905`.
- Search for the `code` at [polygons](http://polygons.openstreetmap.fr/).
- For our purposes, GeoJSONs were selected according to the following criteria: (1) sparsity of vertices (`NPoints`) and (2) accuracy of shape.

In [17]:
# new york city, ny
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/nyc.txt') as response:
    nyc_json = json.load(response)

# kansas city, mo
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kcm.txt') as response:
    kcm_json = json.load(response)

# joplin, mo
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/jm.txt') as response:
    jm_json = json.load(response)

# oglala lakota county, nd
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/olsd.txt') as response:
    olsd_json = json.load(response)

# kusilvak census area, ak
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kca.txt') as response:
    kca_json = json.load(response)

In [18]:
add_to_json_dict = {
    '02158':{
        'properties':{
            'CENSUSAREA':17_081.43,
            'NAME':'Kusilvak Census Area'
        }, 'geometry':asShape(kca_json).buffer(0)
    }, 
    '46102':{
        'properties':{
            'CENSUSAREA':2_093.90,
            'NAME':'Oglala Lakota'
        }, 'geometry':asShape(olsd_json).buffer(0)
    },
    '29JOP':{
        'properties':{
            'CENSUSAREA':35.56,
            'NAME':'Joplin'
        }, 'geometry':asShape(jm_json).buffer(0)
    },
    '29KAN':{
        'properties':{
            'CENSUSAREA':314.95,
            'NAME':'Kansas City'
        }, 'geometry':asShape(kcm_json).buffer(0)
    },
    '36NYC':{
        'properties':{
            'CENSUSAREA':302.64,
            'NAME':'New York City'
        }, 'geometry':asShape(nyc_json).buffer(0)
    }
}

In [19]:
for fips in ['02158', '46102', '29KAN', '29JOP', '36NYC']:
    county_json['features'].append(
        Feature(id=fips,
                geometry=add_to_json_dict[fips]['geometry'],
                properties=add_to_json_dict[fips]['properties'])
    )

In [20]:
with open('data/county_json.json', 'w') as f:
    json.dump(county_json, f)

## add centroid latitude and longitude coordinates and county area from `county_json` to `pop_df`

We will use `shapely` to calculate the [centroid](https://en.wikipedia.org/wiki/Centroid) coordinates for the counties (in case we wish to plot bubble maps).

In [21]:
def centroid(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            polygon = asShape(d['geometry']).buffer(0)
            return polygon.centroid.coords[0]
        
def county_area(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            return d['properties']['CENSUSAREA']

In [22]:
pop_df['area'] = pop_df['fips'].apply(county_area)
pop_df['lon'], pop_df['lat'] = zip(*pop_df['fips'].apply(centroid).to_list())
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,lat
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108,2238.549,-107.681649,43.904771
3141,Wyoming,Weston,6927,56045,6236,45,113,285,2398.089,-104.567290,43.839661
3142,New York,New York City,8336817,36NYC,2681976,1825848,1228598,2423590,302.640,-73.939368,40.663516
3143,Missouri,Kansas City,495327,29KAN,297691,143644,13373,50523,314.950,-94.554422,39.127195
3144,Missouri,Joplin,50925,29JOP,44610,1629,967,2444,35.560,-94.505664,37.078985


# 3. add 2016 general election data

Mask compliance has been very political, so it would be interesting to see how political differences vary by county. Data taken from [github.com/tonmcg](https://github.com/tonmcg). Alaska data taken from [RRH Elections](https://rrhelections.com/index.php/2018/02/02/alaska-results-by-county-equivalent-1960-2016/).

In [23]:
with urlopen('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv') as response:
    elect_df = pd.read_csv(
        response,
        encoding='latin-1',        # to avoid unicode error
        dtype={
            'votes_dem':'int',
            'votes_gop':'int',
            'total_votes':'int',
            'combined_fips':'str'},
        index_col=0
    )
elect_df.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [24]:
elect_df.rename(
    columns={
        'county_name':'county',
        'combined_fips':'fips',
    }, inplace=True
)

elect_df = remove_county_terms(elect_df, 'county')
elect_df['fips'] = elect_df['fips'].apply('{0:0>5}'.format)        # https://stackoverflow.com/a/23836353
elect_df = elect_df[['state_abbr', 'county', 'fips', 'votes_dem', 'votes_gop', 'total_votes']]
elect_df = elect_df.sort_values(by='fips')
elect_df.head()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
29,AL,Autauga,01001,5908,18110,24661
30,AL,Baldwin,01003,18409,72780,94090
31,AL,Barbour,01005,4848,5431,10390
32,AL,Bibb,01007,1874,6733,8748
33,AL,Blount,01009,2150,22808,25384


## add New York City, Kansas City, and Joplin election data

In [25]:
nyc_elect_df = elect_df[(elect_df['state_abbr'] == 'NY') & (elect_df['county'] == boroughs[0])].select_dtypes(include='number')

for b in boroughs[1:]:
    nyc_elect_df += elect_df[(elect_df['state_abbr'] == 'NY') & (elect_df['county'] == b)].select_dtypes(include='number').values
    
nyc_elect_df

,votes_dem,votes_gop,total_votes
1982,1969920,461174,2490750


In [ ]:
# nyc_elect_df['per_dem'] = nyc_elect_df['votes_dem'] / nyc_elect_df['total_votes']
# nyc_elect_df['per_gop'] = nyc_elect_df['votes_gop'] / nyc_elect_df['total_votes']

In [26]:
nyc_elect_df['state_abbr'] = 'NY'
nyc_elect_df['county'] = 'New York City'
nyc_elect_df['fips'] = '36NYC'
nyc_elect_df.head()

,votes_dem,votes_gop,total_votes,state_abbr,county,fips
1982,1969920,461174,2490750,NY,New York City,36NYC


In [27]:
# https://en.wikipedia.org/wiki/2016_United_States_presidential_election_in_Missouri

mo_elect_df = pd.DataFrame(
    [['MO', 'Kansas City', '29KAN', 97735, 24654, 128601],
     ['MO', 'Joplin', '29JOP', np.nan, np.nan, np.nan]]
    , columns=elect_df.columns)
mo_elect_df

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
0,MO,Kansas City,29KAN,97735.0,24654.0,128601.0
1,MO,Joplin,29JOP,NaN,NaN,NaN


In [28]:
elect_df = elect_df.append([nyc_elect_df, mo_elect_df], ignore_index=True)
elect_df.tail()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
3139,WY,Washakie,56043,532.0,2911.0,3715.0
3140,WY,Weston,56045,294.0,2898.0,3334.0
3141,NY,New York City,36NYC,1969920.0,461174.0,2490750.0
3142,MO,Kansas City,29KAN,97735.0,24654.0,128601.0
3143,MO,Joplin,29JOP,NaN,NaN,NaN


## add alaska elections data

Data taken from [RRH Elections](https://rrhelections.com/index.php/2018/02/02/alaska-results-by-county-equivalent-1960-2016/).

In [29]:
ak_elect_df = pd.read_excel('data/2016 AK Gen Official.xlsx', sheet_name='By CE')
ak_elect_df = ak_elect_df.iloc[0:29, 0:12]
ak_elect_df.head()

,ED/Muni,Municipality Code,Registered Voters,Times Counted,"Castle, Darrell L.","Clinton, Hillary","De La Fuente, Roque","Johnson, Gary","Stein, Jill","Trump, Donald J.",Write-in 60,ED Total
0,Ketchikan Gateway,Ketchikan,10512,4283,48,1295,13,339,84,2354,104,4237
1,Prince of Wales-Hyder,Prince of Wales-Hyder,4630,1831,67,666,29,93,65,831,59,1810
2,Sitka,Sitka,7218,2787,38,1261,18,145,78,1146,73,2759
3,Petersburg,Petersburg,2741,1078,12,334,7,64,37,577,32,1063
4,Wrangell,Wrangell,1731,764,7,177,3,35,13,512,13,760


In [30]:
ak_elect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ED/Muni               29 non-null     object
 1   Municipality Code     29 non-null     object
 2   Registered Voters     29 non-null     object
 3   Times Counted         29 non-null     object
 4   Castle, Darrell L.    29 non-null     object
 5   Clinton, Hillary      29 non-null     object
 6   De La Fuente, Roque   29 non-null     object
 7   Johnson, Gary         29 non-null     object
 8   Stein, Jill           29 non-null     object
 9   Trump, Donald J.      29 non-null     object
 10  Write-in 60           29 non-null     object
 11  ED Total              29 non-null     object
dtypes: object(12)
memory usage: 2.8+ KB


In [31]:
ak_elect_df.rename(
    columns={
        'Trump, Donald J. ':'votes_gop',
        'Clinton, Hillary ':'votes_dem'
    }, inplace=True
)
ak_elect_df = ak_elect_df[['ED/Muni', 'votes_gop', 'votes_dem', 'ED Total']].sort_values(by='ED/Muni')
ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']] = ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']].astype(int)
ak_elect_df = ak_elect_df.sort_values(by='ED/Muni')
ak_elect_df.head()

,ED/Muni,votes_gop,votes_dem,ED Total
22,Aleutians East,198,121,369
24,Aleutians West,260,493,846
19,Anchorage,39942,32130,81678
12,Bethel,809,2178,3933
25,Bristol Bay,180,99,316


In [32]:
print(len(ak_elect_df))
print(len(elect_df[elect_df['state_abbr'] == 'AK']))

29
29


In [35]:
elect_df.loc[elect_df['state_abbr'] == 'AK', ['votes_gop', 'votes_dem', 'total_votes']] = ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']].values
elect_df.tail()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
3139,WY,Washakie,56043,532.0,2911.0,3715.0
3140,WY,Weston,56045,294.0,2898.0,3334.0
3141,NY,New York City,36NYC,1969920.0,461174.0,2490750.0
3142,MO,Kansas City,29KAN,97735.0,24654.0,128601.0
3143,MO,Joplin,29JOP,NaN,NaN,NaN


In [36]:
elect_df[elect_df['state_abbr'] == 'AK'].head()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
67,AK,Alaska,02013,121.0,198.0,369.0
68,AK,Alaska,02016,493.0,260.0,846.0
69,AK,Alaska,02020,32130.0,39942.0,81678.0
70,AK,Alaska,02050,2178.0,809.0,3933.0
71,AK,Alaska,02060,99.0,180.0,316.0


In [37]:
pop_df = pop_df.merge(elect_df[['fips', 'votes_gop', 'votes_dem', 'total_votes']], on='fips', how='left')

# 4. add income data

Median income statistics taken from [data.census.gov](https://data.census.gov/cedsci/table?q=s1901&tid=ACSST1Y2018.S1901) (2017 ACS 1-Year Estimates). According to the data table, the median household income is reported in column `S1901_C01_012E`.

In [38]:
income_df = pd.read_csv('data/ACSST5Y2018.S1901_data_with_overlays_2020-07-16T134009.csv',
                        usecols=['GEO_ID', 'NAME', 'S1901_C01_012E'])
income_df = income_df.drop(0, axis=0)
income_df.rename(
    columns={
        'GEO_ID':'fips',
        'S1901_C01_012E':'median_income'
    }, inplace=True
)
income_df['median_income'] = income_df['median_income'].astype(float)
income_df['fips'] = income_df['fips'].str[-5:]
income_df.head()

,fips,NAME,median_income
1,01001,"Autauga County, Alabama",58786.0
2,01003,"Baldwin County, Alabama",55962.0
3,01005,"Barbour County, Alabama",34186.0
4,01007,"Bibb County, Alabama",45340.0
5,01009,"Blount County, Alabama",48695.0


In [39]:
income_df[income_df['median_income'].isna()]

,fips,NAME,median_income
1817,35039,"Rio Arriba County, New Mexico",NaN


Rio Arriba Income statistics taken from [datausa.io](https://datausa.io/profile/geo/rio-arriba-county-nm#:~:text=Median%20household%20income%20in%20Rio%20Arriba%20County%2C%20NM%20is%20%2433%2C422.)

In [40]:
income_df.at[income_df['fips'] == '35039', 'median_income'] = 33_422

In [41]:
income_df['county'], income_df['state'] = zip(*income_df['NAME'].str.split(', ').tolist())
income_df = income_df.drop('NAME', axis=1)
income_df = remove_county_terms(income_df, 'county')
income_df['median_income'] = income_df['median_income'].astype(int)
income_df.head()

,fips,median_income,county,state
1,01001,58786,Autauga,Alabama
2,01003,55962,Baldwin,Alabama
3,01005,34186,Barbour,Alabama
4,01007,45340,Bibb,Alabama
5,01009,48695,Blount,Alabama


In [42]:
# only an estimate of median income

nyc_income = income_df[(income_df['state'] == 'New York') & (income_df['county'] == boroughs[0])]['median_income'].values \
                * pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == boroughs[0])]['total_pop'].values
nyc_pop = pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == boroughs[0])]['total_pop'].values

for b in boroughs[1:]:
    nyc_income += income_df[(income_df['state'] == 'New York') & (income_df['county'] == b)]['median_income'].values \
                     * pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == b)]['total_pop'].values
    nyc_pop += pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == b)]['total_pop'].values
    
nyc_income = nyc_income / nyc_pop
nyc_income[0]

61884.61918031786

In [43]:
income_df2 = pd.DataFrame(
    [['36NYC', int(nyc_income[0]), 'New York City', 'New York'],
     ['29KAN', 52405, 'Kansas City', 'Missouri'],
     ['29JOP', 42782, 'Joplin', 'Missouri']]
    , columns=income_df.columns)
income_df2

,fips,median_income,county,state
0,36NYC,61884,New York City,New York
1,29KAN,52405,Kansas City,Missouri
2,29JOP,42782,Joplin,Missouri


In [44]:
income_df = income_df.append(income_df2, ignore_index=True)
income_df.tail(5)

,fips,median_income,county,state
3218,72151,16013,Yabucoa Municipio,Puerto Rico
3219,72153,14954,Yauco Municipio,Puerto Rico
3220,36NYC,61884,New York City,New York
3221,29KAN,52405,Kansas City,Missouri
3222,29JOP,42782,Joplin,Missouri


In [45]:
pop_df = pop_df.merge(income_df[['fips', 'median_income']], on='fips', how='left')
pop_df.head()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,lat,votes_gop,votes_dem,total_votes,median_income
0,Alabama,Autauga,55869,01001,41215,11098,646,1671,594.436,-86.641196,32.536153,18110.0,5908.0,24661.0,58786
1,Alabama,Baldwin,223234,01003,185747,19215,2346,10534,1589.784,-87.723954,30.725862,72780.0,18409.0,94090.0,55962
2,Alabama,Barbour,24686,01005,11235,11807,116,1117,884.876,-85.389245,31.867889,5431.0,4848.0,10390.0,34186
3,Alabama,Bibb,22394,01007,16663,4719,46,623,622.582,-87.124963,32.996456,6733.0,1874.0,8748.0,45340
4,Alabama,Blount,57826,01009,50176,872,163,5582,644.776,-86.569756,33.985248,22808.0,2150.0,25384.0,48695


# 5. add educational attainment data

Educational attainment statistics taken from [data.census.gov](https://data.census.gov/cedsci/table?tid=ACSST5Y2018.S1501&g=0400000US04) (2017 ACS 1-Year Estimates).

- `S1501_C01_006E` -- population > 25yo
- `S1501_C01_007E` -- less than 9th grade
- `S1501_C01_008E` -- some high school
- `S1501_C01_009E` -- high school or GED
- `S1501_C01_010E` -- some college
- `S1501_C01_011E` -- associate's
- `S1501_C01_012E` -- bachelor's
- `S1501_C01_013E` -- graduate or professional

In [46]:
edu_cols = ['S1501_C01_'+f'{i:03d}'+'E' for i in range(6,14)]
edu_col_names = ['pop25', 'no_hs', 'some_hs', 'hs', 'some_college', 'associates', 'bachelors', 'graduate']
edu_dict = dict(zip(edu_cols, edu_col_names))
edu_dict.update({'GEO_ID':'fips'})

edu_df = pd.read_csv('data/ACSST5Y2018.S1501_data_with_overlays_2020-07-18T170455.csv',
                     usecols=['GEO_ID', 'NAME']+edu_cols)
edu_df = edu_df.drop(0, axis=0)
for col in edu_cols:
    edu_df[col] = edu_df[col].astype(int)
edu_df.rename(
    columns=edu_dict,
    inplace=True
)
edu_df['fips'] = edu_df['fips'].str[-5:]
edu_df.head()

,fips,NAME,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate
1,01001,"Autauga County, Alabama",37166,956,3248,12119,7554,2998,5903,4388
2,01003,"Baldwin County, Alabama",146989,3978,10332,40579,32266,13759,30431,15644
3,01005,"Barbour County, Alabama",18173,1490,3411,6486,3287,1279,1417,803
4,01007,"Bibb County, Alabama",15780,903,1747,7471,2938,908,1197,616
5,01009,"Blount County, Alabama",39627,2967,4894,13489,8492,4775,3217,1793


In [47]:
edu_df['county'], edu_df['state'] = zip(*edu_df['NAME'].str.split(', ').tolist())
edu_df = edu_df.drop('NAME', axis=1)
edu_df = remove_county_terms(edu_df, 'county')
edu_df.head()

,fips,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,county,state
1,01001,37166,956,3248,12119,7554,2998,5903,4388,Autauga,Alabama
2,01003,146989,3978,10332,40579,32266,13759,30431,15644,Baldwin,Alabama
3,01005,18173,1490,3411,6486,3287,1279,1417,803,Barbour,Alabama
4,01007,15780,903,1747,7471,2938,908,1197,616,Bibb,Alabama
5,01009,39627,2967,4894,13489,8492,4775,3217,1793,Blount,Alabama


In [48]:
nyc_edu_df = edu_df[(edu_df['state'] == 'New York') & (edu_df['county'] == boroughs[0])].select_dtypes(include='number')

for b in boroughs[1:]:
    nyc_edu_df += edu_df[(edu_df['state'] == 'New York') & (edu_df['county'] == b)].select_dtypes(include='number').values

nyc_edu_df['state'] = 'New York'
nyc_edu_df['county'] = 'New York City'
nyc_edu_df['fips'] = '36NYC'
nyc_edu_df.head()

,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,state,county,fips
1831,5923498,565345,523873,1421617,815961,379457,1292814,924431,New York,New York City,36NYC


In [49]:
edu_df = edu_df.append(nyc_edu_df, ignore_index=True)
edu_df.tail()

,fips,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,county,state
3216,72147,6282,1096,480,2344,487,580,933,362,Vieques Municipio,Puerto Rico
3217,72149,15288,2599,1134,5616,1648,1218,2545,528,Villalba Municipio,Puerto Rico
3218,72151,23916,4975,2245,5972,3636,2645,3706,737,Yabucoa Municipio,Puerto Rico
3219,72153,25976,4977,2259,8182,2381,1791,4902,1484,Yauco Municipio,Puerto Rico
3220,36NYC,5923498,565345,523873,1421617,815961,379457,1292814,924431,New York City,New York


In [50]:
mo_edu_df = pd.DataFrame(
    [['29KAN', 325065, 11373, 22302, 82996, 73203, 23673, 69682, 41836, 'Kansas City', 'Missouri'],
     ['29JOP', 33571, 779, 2580, 10582, 8462, 2576, 5759, 2833, 'Joplin', 'Missouri']]
    , columns=edu_df.columns)
mo_edu_df

,fips,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,county,state
0,29KAN,325065,11373,22302,82996,73203,23673,69682,41836,Kansas City,Missouri
1,29JOP,33571,779,2580,10582,8462,2576,5759,2833,Joplin,Missouri


In [51]:
edu_df = edu_df.append(mo_edu_df, ignore_index=True)
edu_df.tail()

,fips,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate,county,state
3218,72151,23916,4975,2245,5972,3636,2645,3706,737,Yabucoa Municipio,Puerto Rico
3219,72153,25976,4977,2259,8182,2381,1791,4902,1484,Yauco Municipio,Puerto Rico
3220,36NYC,5923498,565345,523873,1421617,815961,379457,1292814,924431,New York City,New York
3221,29KAN,325065,11373,22302,82996,73203,23673,69682,41836,Kansas City,Missouri
3222,29JOP,33571,779,2580,10582,8462,2576,5759,2833,Joplin,Missouri


In [52]:
pop_df = pop_df.merge(edu_df[['fips']+edu_col_names], on='fips', how='left')
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,...,total_votes,median_income,pop25,no_hs,some_hs,hs,some_college,associates,bachelors,graduate
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108,2238.549,-107.681649,...,3715.0,53426,5662,181,409,1717,1434,701,849,371
3141,Wyoming,Weston,6927,56045,6236,45,113,285,2398.089,-104.567290,...,3334.0,52867,5014,129,260,1796,1334,534,676,285
3142,New York,New York City,8336817,36NYC,2681976,1825848,1228598,2423590,302.640,-73.939368,...,2490750.0,61884,5923498,565345,523873,1421617,815961,379457,1292814,924431
3143,Missouri,Kansas City,495327,29KAN,297691,143644,13373,50523,314.950,-94.554422,...,128601.0,52405,325065,11373,22302,82996,73203,23673,69682,41836
3144,Missouri,Joplin,50925,29JOP,44610,1629,967,2444,35.560,-94.505664,...,NaN,42782,33571,779,2580,10582,8462,2576,5759,2833


# 6. add mask usage statistics

In [53]:
with urlopen('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv') as response:
    mask_df = pd.read_csv(response)
mask_df.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [54]:
mask_df.rename(columns={'COUNTYFP':'fips'}, inplace=True)
mask_df['fips'] = mask_df['fips'].apply('{0:0>5}'.format)
mask_df.columns = mask_df.columns.str.lower()
mask_df.head()

,fips,never,rarely,sometimes,frequently,always
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [55]:
def mask_df_averager(fips, custom_fips):
    temp_df = pd.DataFrame(
        [
            np.average(
                mask_df.set_index('fips').loc[fips, :], 
                axis=0, 
                weights=pop_df.set_index('fips').loc[fips, 'total_pop']
            )
        ], columns=mask_df.columns[1:]
    )
    temp_df['fips'] = custom_fips
    return temp_df

In [56]:
nyc_fips = ['36005', '36047', '36061', '36081', '36085']
nyc_mask_df = mask_df_averager(nyc_fips, '36NYC')
nyc_mask_df

,never,rarely,sometimes,frequently,always,fips
0,0.029802,0.022852,0.057863,0.137004,0.75248,36NYC


In [57]:
# estimates from averaging counties
jop_fips = ['29097', '29145']
kan_fips = ['29095', '29047', '29165', '29037']
jop_mask_df = mask_df_averager(jop_fips, '29JOP')
kan_mask_df = mask_df_averager(kan_fips, '29KAN')

In [58]:
mask_df = mask_df.append([nyc_mask_df, jop_mask_df, kan_mask_df], ignore_index=True)
mask_df.tail()

,fips,never,rarely,sometimes,frequently,always
3140,56043,0.204000,0.155000,0.069000,0.285000,0.287000
3141,56045,0.142000,0.129000,0.148000,0.207000,0.374000
3142,36NYC,0.029802,0.022852,0.057863,0.137004,0.752480
3143,29JOP,0.143838,0.141946,0.144730,0.187351,0.383135
3144,29KAN,0.029815,0.058889,0.092879,0.203477,0.614758


In [59]:
pop_df = pop_df.merge(mask_df, on='fips', how='left')
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,...,hs,some_college,associates,bachelors,graduate,never,rarely,sometimes,frequently,always
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108,2238.549,-107.681649,...,1717,1434,701,849,371,0.204000,0.155000,0.069000,0.285000,0.287000
3141,Wyoming,Weston,6927,56045,6236,45,113,285,2398.089,-104.567290,...,1796,1334,534,676,285,0.142000,0.129000,0.148000,0.207000,0.374000
3142,New York,New York City,8336817,36NYC,2681976,1825848,1228598,2423590,302.640,-73.939368,...,1421617,815961,379457,1292814,924431,0.029802,0.022852,0.057863,0.137004,0.752480
3143,Missouri,Kansas City,495327,29KAN,297691,143644,13373,50523,314.950,-94.554422,...,82996,73203,23673,69682,41836,0.029815,0.058889,0.092879,0.203477,0.614758
3144,Missouri,Joplin,50925,29JOP,44610,1629,967,2444,35.560,-94.505664,...,10582,8462,2576,5759,2833,0.143838,0.141946,0.144730,0.187351,0.383135


In [60]:
pop_df['pop_density'] = pop_df['total_pop'] / pop_df['area']
for col in ['white', 'black', 'asian', 'hispanic', 'total_votes']:
    pop_df['per_' + col] = pop_df[col] / pop_df['total_pop']
for col in ['votes_gop', 'votes_dem']:
    pop_df['per_' + col] = pop_df[col] / pop_df['total_votes']
for col in ['no_hs', 'some_hs', 'hs', 'associates', 'bachelors', 'graduate']:
    pop_df['per_' + col] = pop_df[col] / pop_df['pop25']

In [61]:
pop_df['per_college'] = (pop_df['associates'] + pop_df['bachelors'] + pop_df['graduate']) / pop_df['pop25']
pop_df['per_gop'] = pop_df['votes_gop'] / (pop_df['votes_gop'] + pop_df['votes_dem'])
pop_df['gop'] = pop_df['votes_gop'] > pop_df['votes_dem']

In [62]:
pop_df.head()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,...,per_votes_dem,per_no_hs,per_some_hs,per_hs,per_associates,per_bachelors,per_graduate,per_college,per_gop,gop
0,Alabama,Autauga,55869,01001,41215,11098,646,1671,594.436,-86.641196,...,0.239569,0.025722,0.087392,0.326078,0.080665,0.158828,0.118065,0.357558,0.754018,True
1,Alabama,Baldwin,223234,01003,185747,19215,2346,10534,1589.784,-87.723954,...,0.195653,0.027063,0.070291,0.276068,0.093606,0.207029,0.106430,0.407064,0.798123,True
2,Alabama,Barbour,24686,01005,11235,11807,116,1117,884.876,-85.389245,...,0.466603,0.081990,0.187696,0.356903,0.070379,0.077973,0.044186,0.192538,0.528359,True
3,Alabama,Bibb,22394,01007,16663,4719,46,623,622.582,-87.124963,...,0.214220,0.057224,0.110710,0.473447,0.057541,0.075856,0.039037,0.172433,0.782270,True
4,Alabama,Blount,57826,01009,50176,872,163,5582,644.776,-86.569756,...,0.084699,0.074873,0.123502,0.340399,0.120499,0.081182,0.045247,0.246928,0.913855,True


In [63]:
pop_df.isna().sum()

state              0
county             0
total_pop          0
fips               0
white              0
black              0
asian              0
hispanic           0
area               0
lon                0
lat                0
votes_gop          4
votes_dem          4
total_votes        4
median_income      0
pop25              0
no_hs              0
some_hs            0
hs                 0
some_college       0
associates         0
bachelors          0
graduate           0
never              0
rarely             0
sometimes          0
frequently         0
always             0
pop_density        0
per_white          0
per_black          0
per_asian          0
per_hispanic       0
per_total_votes    4
per_votes_gop      4
per_votes_dem      4
per_no_hs          0
per_some_hs        0
per_hs             0
per_associates     0
per_bachelors      0
per_graduate       0
per_college        0
per_gop            4
gop                0
dtype: int64

## save results to csv

In [64]:
dem_df.to_csv('data/dem_df.csv', index=False)
pop_df.to_csv('data/pop_df.csv', index=False)

# Future Work: import Puerto Rico census data

To do:
- find detailed demographic data for Puerto Rico
- find a way to incorporate Puerto Rico into the Altair map

In [ ]:
# with urlopen('https://www2.census.gov/programs-surveys/popest/tables/2010-2019/municipios/totals/prm-est2019-annres.xlsx') as response:
#     pr_df = pd.read_excel(response, header=3)
pr_df = pd.read_excel('data/prm-est2019-annres.xlsx', header=3)
pr_df = pr_df[['Unnamed: 0', 2019]]
pr_df.rename(
    columns={
        'Unnamed: 0':'county',
        2019:'total_pop'
    }, inplace=True
)
pr_df = pr_df[~pr_df['total_pop'].isna()]
pr_df['total_pop'] = pr_df['total_pop'].astype('int')
pr_df.head()

In [ ]:
pr_df['county'] = [s[0] if len(s) > 0 else s for s in pr_df['county'].str.findall("\.([\w\s]+) Municipio\,.+")]
pr_df = pr_df.iloc[1:]          # removing the territory as a whole from the table
pr_df.head()

We also need to add `fips` codes for all of the municipios.

### import Puerto Rico `fips`

In [ ]:
sess = HTMLSession()
res = sess.get('https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county')
table = res.html.find('table.wikitable > tbody > tr')
# puerto rico is fips 72
pr_fips = [[tr.find('td')[1].text, tr.find('td')[0].text] for tr in table[1:] if tr.find('td')[0].text[:2] == '72']
pr_fips_df = pd.DataFrame(pr_fips)
pr_fips_df.rename(
    columns={
        0:'county',
        1:'fips'
    }, inplace=True
)
pr_fips_df.head()

In [ ]:
pr_fips_df['county'] = [s[0] if len(s) > 0 else s for s in pr_fips_df['county'].str.findall("([\w\s]+) Municipality")]
pr_fips_df.head()

In [ ]:
len(list(set(pr_fips_df['county']) - set(pr_df['county'])))

In [ ]:
pr_df = pr_df.merge(pr_fips_df, on='county')
pr_df['state'] = 'Puerto Rico'
pr_df.head()

In [ ]:
pop_df = optimize(pop_df.append(pr_df, ignore_index=True).append(pr_df, ignore_index=True))
pop_df.tail()

## check county names against NYTimes data (again)

In [ ]:
county_diffs = list(set(nyt_df['county']) - set(dem_df['county']))
len(county_diffs)

In [ ]:
county_diffs

The NYTimes dataset is missing diacritical marks in their names. While it would be easier to replace diacritical marks with their "standard" character counterparts, we will preserve them in our final dataframe in the interest of cultural accuracy. This will be handled when we merge `pop_df` with `nyt_df` in the other notebook.